#**Reading the data**

In [1]:
import re
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import numpy as np
from nltk.corpus import stopwords
import os
import warnings
import itertools
# import emoji
from nltk.corpus import stopwords
import string

In [2]:
!gdown --id 1DEiFMHK1ng0G0Z5Lju1e-13ClwQJu1-V

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1DEiFMHK1ng0G0Z5Lju1e-13ClwQJu1-V
To: /content/Political Tweets.xlsx
100% 12.3M/12.3M [00:00<00:00, 90.0MB/s]


In [3]:
data = pd.read_excel("/content/Political Tweets.xlsx")

In [4]:
teeext=data.iloc[11,0]
teeext

'    مكانك علي حدود مش سوق العبور . السيسي اهانه جيش مصر العظيم    '

In [5]:
data

,Text,Year,Source
0,غني الحريه دي أجمل غنوه في الوجوه شمس الح...,2019,Arabic Political Tweets
1,شامم ريحة أحداث يناير,2019,Arabic Political Tweets
2,اكتر من ربع مليون تغريدة ليه ابو لهب هو ولا...,2019,Arabic Political Tweets
3,لا لإهانة اولادنا بيد العرص,2019,Arabic Political Tweets
4,هذا هو حال المصريين في عهد السيسي وعصابته....,2019,Arabic Political Tweets
...,...,...,...
431446,مافيش واحد فالجيش زي خالد الاسلامبولي كدا ...,2019,Arabic Political Tweets
431447,…,2019,Arabic Political Tweets
431448,المستشفيات بتاعتنا وهوا بيعمل قصور وهيعمل ...,2019,Arabic Political Tweets
431449,انا اسف ياأمي حقك وحقنا كلنا هيرجع ان شاء...,2019,Arabic Political Tweets


In [6]:
data['Text'].describe()

count                                                431451
unique                                               128423
top           عاوز 10000 تلاف رتويت هنا وهنوصل تريند عال...
freq                                                   8004
Name: Text, dtype: object

In [7]:
data['Year'].unique()

array([2019])

#**Preproccessing**

#### Data Cleaning & Normlization
>*  Remove Punctuatuions
*  Remove Usernames(mentions)
*   Remove URLs
*   Remove emoji
*   Remove Numbers
*   Remove Special Characters
*   Remove Elongation







In [8]:
!pip install farasapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!git clone https://github.com/aub-mind/arabert

Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 21.28 MiB/s, done.
Resolving deltas: 100% (339/339), done.


In [10]:
!pip install pyarabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 13.4 MB/s 


In [11]:
def removeMention(txt):
  arabic_men=re.sub("@[أ-ي]+","",txt)#//remove Arabic hashtags that have  after it
  arabic_men=re.sub("[أ-ي]@+","",arabic_men) #remove Arabic hashtags that have # before it
  without_men=re.sub("@[A-Za-z]+","",arabic_men)# remove English hashtags
  return without_men

In [12]:
def removeduplicate(txt):
  non_duplicate = ''.join(i for i, _ in itertools.groupby(txt))
  return non_duplicate


In [13]:
def removeDigits(txt):
  without_digit = re.sub('[0-9]+', '', txt) 
  return without_digit

In [14]:
def removeURl(txt):
  without_url= re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', txt)
  return without_url

In [15]:
def removestopwords(txt):
  without_stopwords = ' '.join(word for word in txt.split() if word not in stop_words)
  return without_stopwords

In [16]:
def removepuncatution(txt):

  punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation
  translator = str.maketrans('', '', punctuations)
  without_punc = txt.translate(translator)

  # txtt = re.sub("[\s+\\# ـ!\/_,$%=^*?:@&^~`(+\"]+|[+！，。？、~@￥%……&*（）“ ”:;：；、\\《）《》“”()»〔〕# ]+ ·.『]", "", txt)
  # seq = re.sub("[\s+[`÷×؛<>_()*&% ـ،/:{} ÷×؛<>_()*&^%][ـ،/: ]", "", txtt)
  return without_punc

In [17]:
def removenonrabic(txt):
  # Remove None arabic 
  without_english = re.sub(u"[^\u0621-\u063A\u0640-\u0652 ]", " ", txt)
  return without_english

In [18]:
def removeHashtages(txt):
  arabic_hash=re.sub("#[أ-ي]+","",txt)#//remove Arabic hashtags that have  after it
  arabic_hash_=re.sub("[أ-ي]#+","",arabic_hash) #remove Arabic hashtags that have # before it
  without_hash=re.sub("#[A-Za-z]+","",arabic_hash)# remove English hashtags
  return without_hash

In [19]:
def removetashkeel(txt):
  TATWEEL = u"\u0640"
  p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
  without_tashkeel = re.sub(p_tashkeel,"", txt)
  without_tashkeel=without_tashkeel.replace(TATWEEL, '')
  return without_tashkeel

In [20]:
def remove_Emojis(text):
  emoj = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002500-\U00002BEF"  # chinese char
      u"\U00002702-\U000027B0"
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      u"\U0001f926-\U0001f937"
      u"\U00010000-\U0010ffff"
      u"\u2640-\u2642" 
      u"\u2600-\u2B55"
      u"\u200d"
      u"\u23cf"
      u"\u23e9"
      u"\u231a"
      u"\ufe0f"  # dingbats
      u"\u3030"
                    "]+", re.UNICODE)
  without_emojy=re.sub(emoj, '', text)
  return without_emojy

In [21]:
def Clean_Egyptain_text(raw_text):

  raw_text=raw_text.strip()
  #1 remove duplichated chars in the raw text
  text=removeduplicate(raw_text)

  #2 remove hashtages after remove duplicated chars
  text=removeHashtages(text)

  #3 remove digits
  text= removeDigits(text)

  #4 remove the puncatution from the text 
  text=removepuncatution(text)

  #5 remove the mention and @ char
  text=removeMention(text)

  #6 remove arabic tashkeel from the text
  text=removetashkeel(text)

  #7 remove Emojies from the text
  text=remove_Emojis(text)

  #8 remove url and links from the text
  text=removeURl(text)

  # #9 remove stopwords from the text
  # text=removestopwords(text)

  #10 remove non rabic words from the text
  text=removenonrabic(text)

  return text

In [22]:
clean = {
    'Tweet': data['Text'].apply(lambda x: Clean_Egyptain_text(str(x)))
    }
Cleaned_data = pd.DataFrame.from_dict(clean)

In [23]:
Cleaned_data

,Tweet
0,غني الحريه دي أجمل غنوه في الوجوه شمس الحريه ا...
1,شام ريحة أحداث يناير
2,اكتر من ربع مليون تغريدة ليه ابو لهب هو ولا ايه
3,لا لإهانة اولادنا بيد العرص
4,هذا هو حال المصرين في عهد السيسي وعصابته لا حو...
...,...
431446,مافيش واحد فالجيش زي خالد الاسلامبولي كدا أناب...
431447,
431448,المستشفيات بتاعتنا وهوا بيعمل قصور وهيعمل قصور
431449,انا اسف ياأمي حقك وحقنا كلنا هيرجع ان شاءاله


#### Check missing data

In [24]:
Cleaned_data.isnull().sum()

Tweet    0
dtype: int64

#### Cleaned data CSV

In [25]:
Cleaned_data.to_csv('clean_dataset.csv')